# Spread of an infectious disease
In our project, we are looking at the spread of a contagious disease in an enclosed space. In this room there are a predefined number of beds with patients in them. First, a patient is infected. The patient can infect the other patients on each side of his bed. Now it can be simulated over an arbitrary period of time how the disease spreads. It is possible to change the conditions of spreading with the help of different parameters. Among other things, the size of the room, the probability of infection or the bed of the first infected person can be changed. It is also possible to include other things in the simulation. For example, it is possible to vaccinate patients or move beds every day. In the following we will go into more detail about each part of the program and at the end we will show some examples of its use.
In the following, we will first explain the meaning of each variable. Then we will take a closer look at all the functions used to simulate the spreading. Next, we will explain how we have animated the simulation and finally we will show examples of how the simulation can be used properly.

Explanation of the variable names:

    c:              Choice of the user which simulations he wants
    n:              Number of rows in the dormitory
    m:              Number of columns in the dormitory
    initial_bed:    Row and column of the initial bed with the disease
    p:              Probability of an infection, between 0 and 1
    k:              Duration of the illness in days
    repetitions:    The number of days you want to simulate
    v:              Number of people vaccinated per day
    bs:             Number of beds swapped per day
    incu:           Duration of the incubation period in days 

## Cellular Automata
In this section we will take a closer look at the different functions we use to simulate the spreading. 
Each field in the grid can have a different value indicating the current status. The following statuses are used:

    0:          Healthy
    1:          Healed
    -1:         Vaccinated
    > 1:        Sick
    > 1 + k:    Sick but not infectious (Incubation period)


First we initialize the grid and the desired size and set all fields to 0. Then we set the desired initial bed to sick and return the new grid.

In [1]:
import numpy as np
import random

def initialize_grid(n, m, initial_bed, k, incu):
    grid = np.zeros((n, m))
    row, col = initial_bed
    grid[row, col] = 1+k+incu
    return grid

Next we have a function to find and return all neighbors of a bed. (top, bottom, left, right)

In [10]:
def get_neighbors(grid, row, col):
    n, m = grid.shape
    neighbors = []
    if row > 0:
        neighbors.append((row - 1, col))  # Top neighbor
    if row < n - 1:
        neighbors.append((row + 1, col))  # Bottom neighbor
    if col > 0:
        neighbors.append((row, col - 1))  # Left neighbor
    if col < m - 1:
        neighbors.append((row, col + 1))  # Right neighbor
    return neighbors

Now we start with the normal method to simulate the spread on one day. We show the evolution of this function and will add more features to the function with each step. As parameters we get the old grid, the probability of infection and the duration of the disease.
First we ask for the size of the grid and make a copy. Then we go through each field individually using two for loops. Then we check if the current field is sick (value greater than 1), if yes we subtract one sick day and search for all neighbors. For each neighbor it is checked if it is still healthy (value is 0) and then a random number is generated. If the neighbor is healthy and the generated number is greater than the value p, the neighbor is infected (value 1 + duration of the disease). Finally the new grid is returned.

In [11]:
def simulate_spread_basic(grid, p, k):
    n, m = grid.shape
    new_grid = np.copy(grid)
    for i in range(n):
        for j in range(m):
            if grid[i, j] > 1:  # Check if the bed is infected or already recovered
                new_grid[i,j] -= 1               
                for neighbor in get_neighbors(grid, i, j):
                    if grid[neighbor] == 0 and random.rand() < p:
                        new_grid[neighbor] = 1+k #set the neighbor ill                
    return new_grid

Now we will add a function that can vaccinate a certain number of healthy people every day. After a vaccination, the respective bed can no longer be infected. The function has two parameters, the number of people to vaccinate and the grid. 
A while loop is used to generate random numbers and try to vaccinate them. When enough vaccinations have been distributed, the new grid is returned.

In [18]:
def vaccinate(v, new_grid):
    n, m = new_grid.shape
    vaccinated = 0
    while(vaccinated < v):
        randi = random.randint(0,n-1)
        randj = random.randint(0, m-1)
        if new_grid[randi, randj] == 0:
            new_grid[randi, randj] = -1
            vaccinated += 1

    return new_grid

Now we will add a function that will swap a certain number of beds with each other every day. The function has two parameters, the number of beds to swap and the grid. 
A While loop is used to generate random numbers to select two beds. Then the values of the two beds are exchanged. When enough beds have been swapped, the new grid is returned.

In [13]:
def bed_swap(bs, new_grid):
    n, m = new_grid.shape
    bedsSwaped = 0
    while(bedsSwaped < bs):
        randi1 = random.randint(0,n-1)
        randj1 = random.randint(0, m-1)
        randi2 = random.randint(0,n-1)
        randj2 = random.randint(0, m-1)
        tmpVal = new_grid[randi1, randj1]
        new_grid[randi1, randj1] = new_grid[randi2, randj2]
        new_grid[randi2, randj2] = tmpVal
        bedsSwaped += 1

    return new_grid

### Final Function
In the final function a parameter is added (incu = The incubation period). The first part of the function remains almost unchanged, with the difference that now it is counted how many healthy beds are left. In addition, before the neighbors of a sick bed can be infected, it is checked if the incubation period is over and if a bed is infected, the incubation period is also added to the value.
Then a check is made to see if there are enough healthy beds left to vaccinate. If not, the value of the beds to be vaccinated is decreased.
Then, if necessary, the two functions to vaccinate and swap beds are called and finally the new grid is returned.

In [14]:
def simulate_spread(grid, p, k, v, bs, incu):
    n, m = grid.shape
    uninfectedBeds = 0
    new_grid = np.copy(grid)
    for i in range(n):
        for j in range(m):
            if grid[i, j] == 0:
                uninfectedBeds += 1
            if grid[i, j] > 1:  # Check if the bed is infected or already recovered
                new_grid[i,j] -= 1               
                for neighbor in get_neighbors(grid, i, j):
                    if new_grid[i,j] <= 1+k:        #check if incubation time is over
                        if grid[neighbor] == 0 and random.rand() < p:
                            new_grid[neighbor] = 1+k+incu #set the neighbor ill  
                            uninfectedBeds -= 1 
    
    if uninfectedBeds < v:
        v = uninfectedBeds

    if v > 0:
        new_grid = vaccinate(v, new_grid)

    if bs > 0:
        new_grid = bed_swap(bs, new_grid)
    
    return new_grid


## Animation

## User Input
In this section we show the user input and the main method. The following five functions are pure user input and therefore we will not explain them in detail. The first function lets the user decide whether to simulate the normal spreading or to use the additional options. The second one asks for all the parameters that are always needed and the last three are called if you want to set the other options as well. 

In [16]:
def get_user_choice():
    c = int(input("Enter the number of your choice:\n1: Normal Execute\n2: Add Vaccination\n3: Add Bed Swap\n4: Add Incubation Period "))
    return c

def get_user_input():
    n = int(input("Enter the number of rows in the dormitory: "))
    m = int(input("Enter the number of columns in the dormitory: "))
    initial_bed = input("Enter the initial infected bed position (in the format 'row,column'): ")
    initial_bed = tuple(map(int, initial_bed.split(',')))
    p = float(input("Enter the probability of infection (0 < p < 1): "))
    k = int(input("Enter the duration of the disease (in days): "))
    repetitions = int(input("Enter the number of days you want to simulate: "))
    return n, m, initial_bed, p, k, repetitions

def get_user_input_vaccines():
    v = int(input("Enter the number of vaccines per day: "))
    return v

def get_user_input_bedSwap():
    bs = int(input("Enter the number of Bed Swaps you want to perform per day: "))
    return bs

def get_user_input_incubation():
    incu = int(input("Enter the duration of the incubation period (in days): "))
    return incu

In the main method the user is first asked which options he wants to configure, then all required parameters are requested from the user. When all parameters are received, the first grid is created. Then a list of grids is created and the first one is copied into the list. Then the spreading is simulated for each day and the newly obtained grids are added to the list. Finally, the list with all grids is displayed using the animation functions.

In [17]:
def main():
    c = get_user_choice()
    n, m, initial_bed, p, k, repetitions = get_user_input()
    incu = 0
    v = 0
    bs = 0
    
    if c > 1:
        v = get_user_input_vaccines()
        if c > 2:
            bs = get_user_input_bedSwap()
            if c > 3:
                incu = get_user_input_incubation()
        

    grid = initialize_grid(n, m, initial_bed, k, incu)
    grid_list = [grid.copy()]
    
    for _ in range(repetitions):
        grid = simulate_spread(grid, p, k, v, bs, incu)
        grid_list.append(grid.copy())

## Some Simulations